<a href="https://colab.research.google.com/github/tejaswishetty17/GenAI_Work/blob/main/PineConeVectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Pinecone Vectore DB

In [ ]:
api_key = ""

In [ ]:
!pip install -qU langchain langchain-pinecone langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 17.2 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=api_key)

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small", dimensions=1024, api_key = "")

In [ ]:
from pinecone import ServerlessSpec

index_name = "rag"

if not pc.has_index(index_name):
  pc.create_index(
      name = index_name,
      dimension=1024,
      metric = "cosine",
      spec = ServerlessSpec(cloud = "aws", region="us-east-1")
  )
index = pc.Index(index_name)

In [ ]:
index

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
vector_store.add_documents(documents=documents)

['72776e7d-c1f2-4919-86e8-f4dd629aed06',
 '9ca39926-c2f6-4c7d-9314-bcd3e82dd283',
 '15aa6a76-d484-4e9d-a108-bcb862f7bbe1',
 '9f533e13-256d-4238-ae3b-84d138e37056',
 '3cd41966-d4d2-422b-a136-48539743f386',
 '8dcae599-c3d7-47f2-95bd-a3707ba77271',
 '41e75506-dfcf-4dc0-bd97-0a3501b0921a',
 'c681f513-fd09-45aa-9b99-8e7be794423f',
 'e54db41a-a310-4459-9a79-d54b1e80d554',
 'e6418730-bf52-40cf-b18e-b4463a5ba326']

In [ ]:
###Query Directly
results = vector_store.similarity_search(
    "Langchain provides abstractions to make working with LLMs easy",
    k=2,
    filter = {"source":"tweet"},
)

for res in results:
  print(f" * {res.page_content} [{res.metadata}]")

 * LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
 * Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
###Query Directly
results = vector_store.similarity_search_with_score(
    "will it be hot tomorrow?",
    k=1,
    filter = {"source":"news"},
)

for res,score in results:
  print(f" *[SIM={score:3f}] {res.page_content} [{res.metadata}]")

 *[SIM=0.563430] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},

)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(id='9f533e13-256d-4238-ae3b-84d138e37056', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]